In [2]:
# Dynamically add project root to sys.path
import sys
import os
sys.path.append(os.getcwd())
from data_pipelines.daily_pipeline import run_daily_pipeline
from data_pipelines.save_to_db import save_all_to_db
import pandas as pd
%load_ext autoreload
%autoreload 2


In [3]:
movies_df = pd.read_pickle('data_pipelines/temp_data/movies.pkl')

In [7]:
screenings_df, cinemas_df = run_daily_pipeline()

save_all_to_db(cinemas_df, movies_df, screenings_df)

INFO:root:Starting daily data pipeline...
INFO:WDM:====== WebDriver manager ======
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Driver [/Users/ardsnijders/.wdm/drivers/chromedriver/mac64/133.0.6943.141/chromedriver-mac-x64/chromedriver] found in cache


/Users/ardsnijders/Documents/cineville_scraper/backend


In [2]:
from data_pipelines.save_to_db import get_existing_movies

movies = get_existing_movies()

In [3]:
movies

{('10 things i hate about you', 1999),
 ('a complete unknown', 2024),
 ('a real pain', 2024),
 ('about dry grasses', 2023),
 ('all we imagine as light', 2024),
 ('alpha', 2024),
 ('als de kraanvogels overvliegen', 1957),
 ('amal', 2023),
 ('anora', 2024),
 ('attack on titan: the last attack', 2024),
 ('babygirl', 2024),
 ('becoming led zeppelin', 2025),
 ('before sunrise', 1995),
 ('before sunset', 2004),
 ('belle de jour', 1967),
 ('blue velvet', 1986),
 ('bottoms', 2023),
 ('bridget jones: mad about the boy', 2025),
 ('call me agnes', 2024),
 ('captain america: brave new world 2d', 2025),
 ('caught by the tides', 2024),
 ('climates', 2006),
 ('companion', 2025),
 ('conclave', 2024),
 ('confidenza', 2023),
 ('dahomey', 2024),
 ('david lynch: the art life', 2016),
 ('de eeuw van simeon ten holt', 2025),
 ('dead poets society', 1989),
 ('den of thieves: pantera', 2025),
 ('diamant brut', 2024),
 ('dikkie dik 2: een nieuwe vriend voor dikkie dik', 2024),
 ('dragon ball z: battle of gods'

### Pipeline, step by step

In [17]:
import logging
import pandas as pd
from data_pipelines.scrapers.filmladder import FilmladderScraper
from data_pipelines.daily_pipeline import assign_ids_cinemas, assign_ids_screenings, extract_unique_movies
from data_pipelines.daily_pipeline import add_cineville_tag, add_imdb_links, clean_screenings, get_new_movies
from data_pipelines.save_to_db import save_movies, save_screenings, save_cinemas
# from external.imdb_api import fetch_imdb_metadata
from data_pipelines.save_to_db import get_existing_movies

%load_ext autoreload
%autoreload 2

logging.basicConfig(level=logging.INFO)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [112]:
logging.info("Starting daily data pipeline...")

# 1️⃣ Scrape Filmladder (returns two DataFrames)
filmladder = FilmladderScraper()
screenings_df, cinemas_df = filmladder.run()

INFO:root:Starting daily data pipeline...
INFO:WDM:====== WebDriver manager ======
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Driver [/Users/ardsnijders/.wdm/drivers/chromedriver/mac64/133.0.6943.141/chromedriver-mac-x64/chromedriver] found in cache


<html lang="nl"><head>
    <!-- Google tag (gtag.js) -->
    <script async="" src="//c.amazon-adsystem.com/aax2/apstag.js"></script><script async="" src="//resources.planetnine.com/player/latest/Pnvp.js?t=0.3149822278312746&amp;h=www.filmladder.nl"></script><script async="" src="https://www.googletagmanager.com/gtag/js?id=G-FXGH1XSDLV"></script>
    <script>
      window.dataLayer = window.dataLayer || [];
      function gtag(){dataLayer.push(arguments);}
      gtag('js', new Date());

      gtag('config', 'G-FXGH1XSDLV');
    </script>

    <meta charset="utf-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width">
    <meta name="theme-color" content="#FFCC02">

    <link rel="apple-touch-icon-precomposed" type="image/png" href="/assets/apple-touch-icon-144x144-af857028c3108d31c1be3c58e48fb488b2aee4fe36160f5a674fa708427176df.png" sizes="144x144">
    <link rel="apple-touch-icon-precomposed" type="image/png" href="/assets/ap

In [11]:
# 2️⃣ Assign IDs
screenings_df = assign_ids_screenings(screenings_df)
screenings_df = clean_screenings(screenings_df)
cinemas_df = add_cineville_tag(assign_ids_cinemas(cinemas_df))

/Users/ardsnijders/Documents/cineville_scraper/backend


In [45]:
scraped_movies = extract_unique_movies(screenings_df)

In [47]:
new_movies = get_new_movies(movies_df)
new_movies


In [52]:
new_movies

,movie_id,title,year,movie_link
18,8d5dda8799,how to make millions before grandma dies,2024,https://www.filmladder.nl/film/how-to-make-mil...
21,b2a0458b81,looney tunes: the day the earth blew up (ov),2024,https://www.filmladder.nl/film/looney-tunes-th...
24,5a5e116411,mufasa: the lion king (ov),2024,https://www.filmladder.nl/film/mufasa-the-lion...
27,04839d5c56,paddington in peru (ov),2024,https://www.filmladder.nl/film/paddington-in-p...
37,c300e00de1,born in flames,1983,https://www.filmladder.nl/film/born-in-flames-...
38,c3da9655b4,chungking express,None,https://www.filmladder.nl/film/chungking-expre...
56,dad590daa2,de brief die nooit verzonden werd,1960,https://www.filmladder.nl/film/de-brief-die-no...
71,271defa604,the wild pear tree,None,https://www.filmladder.nl/film/the-wild-pear-t...
101,452acb4a5b,inland empire,2006,https://www.filmladder.nl/film/inland-empire-2...
120,d44c151de2,last breath,2024,https://www.filmladder.nl/film/last-breath-202...


In [108]:
from data_pipelines.scrapers.imdb import IMDBScraper
scraper = IMDBScraper(headless=True)
new_movies = scraper.run(new_movies)

INFO:WDM:====== WebDriver manager ======
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Driver [/Users/ardsnijders/.wdm/drivers/chromedriver/mac64/133.0.6943.141/chromedriver-mac-x64/chromedriver] found in cache
Scraping Progress: 100%|██████████| 26/26 [00:50<00:00,  1.94s/it]


In [109]:
new_movies = new_movies[~new_movies["imdb_link"].isin(set(existing_movies["imdb_link"].dropna()))]



In [111]:
save_movies(new_movies)

In [ ]:
# 3️⃣ Extract and fetch IMDb metadata for unique movies
# TODO: first reference movies_df with existing movies sqlite backend.
# only fetch IMDB links for new titles.
# movies_df = extract_unique_movies(screenings_df)

# movies_df = fetch_metadata(movies_df)

# # 4️⃣ Store data in the database
# save_movies(movies_df)
# save_screenings(screenings_df)
# save_cinemas(cinemas_df)

logging.info("Daily data pipeline completed.")